In [1]:
import numpy as np # linear algebra
import pandas as pd
import nltk
import string
from nltk.corpus import stopwords
from nltk import sent_tokenize, word_tokenize, TweetTokenizer 
from nltk import word_tokenize
from nltk.stem import WordNetLemmatizer
import re
wordnet_lemmatizer = WordNetLemmatizer()
import spacy
nlp = spacy.load('en_core_web_sm')
from afinn import Afinn
af = Afinn()

In [2]:
preprocessed =pd.read_csv('../Data/feature.csv')
truth =pd.read_csv('../Data/ground_truth.csv')
truth = np.array(truth.iloc[:,0])

In [3]:
data=[]
columns=['POS-2','POS-1','POS','POS+1','POS+2']
for index, row in preprocessed.iterrows():
    text=row['POS'].split()
    for t in range(len(text)):
        value=[]
        for i in range(t-2,t+3):
            if((i<0)|(i>=len(text))):
                value.append("None")
            else:
                value.append(text[i])
        zipped = zip(columns, value)
        a_dictionary = dict(zipped)
        data.append(a_dictionary)
POS = pd.DataFrame.from_dict(data)

In [4]:
data=[]
columns=['id','text', 'positive', 'strong_subjectives', 'implicatives', 'npov_word',
         'factives', 'assertives', 'weak_subjectives', 'hedges', 'entailment',
         'negative', 'report_verbs', 'lemma', 'position', 'grammatical','polarity']
for index, row in preprocessed.iterrows():
    text=row['text'].split()
    for t in range(len(text)):
        value=[]
        value.append(index)
        for column in columns[1:]:
            value.append(row[column].split()[t])
        zipped = zip(columns, value)
        a_dictionary = dict(zipped)
        data.append(a_dictionary)

word_values = pd.DataFrame.from_dict(data)

In [5]:
data=[]
columns=['positive', 'strong_subjectives', 'implicatives','factives', 'assertives',
         'weak_subjectives', 'hedges', 'entailment','negative', 'report_verbs']
col=[i+"_around" for i in columns]
for index, row in preprocessed.iterrows():
    text=row['text'].split()
    for t in range(len(text)):
        value=[]
        for column in columns:
            p=[row[column].split()[i] for i in range(t-2,t+3) if (i>=0)&(i<len(text))&(i!=t)]
            value.append("True" if "True" in p else "False")        
        zipped = zip(col, value)
        a_dictionary = dict(zipped)
        data.append(a_dictionary)

word_surrounding = pd.DataFrame.from_dict(data)

In [6]:
data=[]
for index, row in preprocessed.iterrows():
    text=row['text'].split()
    for t in range(len(text)):
        value = [1 if (truth[index]==text[t]) else 0]        
        data.append(value)

label = pd.DataFrame.from_dict(data)
label.columns=['label']

In [7]:
trainset = pd.concat([word_values,POS, word_surrounding,label], axis=1, sort=False)

In [8]:
word=trainset['text']
lemma=trainset['lemma']
train= trainset.drop(['text','lemma'],axis=1)

In [9]:
import gensim
path = "../Data/GoogleNews-vectors-negative300.bin"
model = gensim.models.KeyedVectors.load_word2vec_format(path,binary=True)

In [10]:
word_vec=[]
col=["word_vec_"+str(i) for i in range(300)]

for i in range(len(word)):
    
    if word[i] in model.vocab:
        w = model[word[i]]
    else:
        w=np.zeros(300)
    
    zipped = zip(col, w)
    a_dictionary = dict(zipped)
    word_vec.append(a_dictionary)


In [11]:
word_lemma=[]
col=["lemma_vec_"+str(i) for i in range(300)]

for i in range(len(lemma)):
    if lemma[i] in model.vocab:
        w = model[lemma[i]]
    else:
        w=np.zeros(300)

    zipped = zip(col, w)
    a_dictionary = dict(zipped)
    word_lemma.append(a_dictionary)

In [12]:
word_vec=pd.DataFrame(word_vec)
word_lemma=pd.DataFrame(word_lemma)
word=pd.DataFrame(word)

In [13]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
train_encoded = train.apply(le.fit_transform)

In [14]:
train_encoded =pd.concat([word,word_vec,word_lemma,train_encoded],axis=1)

In [15]:
len(train_encoded)

1340980

In [16]:
import pickle
train_encoded.to_pickle('../Data/train_encoded.pickle')
# df2 = pd.read_pickle('my_df.pickle')